In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicta-1-0-predict-the-unpredictable/submission_key.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable/sample_submission.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable/historical_weather.csv


In [2]:
df = pd.read_csv("/kaggle/input/predicta-1-0-predict-the-unpredictable/historical_weather.csv")

In [3]:
print(df)

       city_id        date  avg_temp_c  min_temp_c  max_temp_c  \
0         C001  2014-01-01         6.6        -1.4        11.6   
1         C001  2014-01-02         9.3         6.3        13.3   
2         C001  2014-01-03         7.6         1.9        14.0   
3         C001  2014-01-04         7.6         3.9        13.3   
4         C001  2014-01-05         8.6         0.5        16.9   
...        ...         ...         ...         ...         ...   
182333    C112  2018-12-27        22.0        15.9        27.9   
182334    C112  2018-12-28        21.9        14.9        28.2   
182335    C112  2018-12-29        22.4        16.3        28.2   
182336    C112  2018-12-30        21.6        18.5        26.6   
182337    C112  2018-12-31        20.2        17.1        25.0   

        precipitation_mm  snow_depth_mm  avg_wind_dir_deg  avg_wind_speed_kmh  
0                    NaN            NaN             168.0                 6.2  
1                    NaN            NaN        

In [4]:
null_pres = df.apply(pd.isnull).sum()/df.shape[0]

In [5]:
null_pres

city_id               0.000000
date                  0.000000
avg_temp_c            0.006713
min_temp_c            0.032281
max_temp_c            0.041094
precipitation_mm      0.382498
snow_depth_mm         0.932883
avg_wind_dir_deg      0.194112
avg_wind_speed_kmh    0.123244
dtype: float64

In [6]:
grouped = df.groupby('city_id')

In [7]:
dataframe_dict = {category: group for category, group in grouped}

In [8]:
null_per = {}
for city in dataframe_dict:
    null_per[city] = {}
    for col in dataframe_dict[city].columns:
        null_per[city][col] = dataframe_dict[city][col].isnull().sum()/dataframe_dict[city].shape[0]

In [9]:
comp_features = ['city_id','date','avg_temp_c','min_temp_c','max_temp_c']
dyn_features = ['precipitation_mm','snow_depth_mm','avg_wind_dir_deg','avg_wind_speed_kmh']
feature_df_dict = {}
for f in dyn_features:
    feature_df_dict[f] = []
    for city in null_per:
        if null_per[city][f]<0.05:
            feature_df_dict[f].append(city)

In [10]:
print(feature_df_dict)

{'precipitation_mm': ['C004', 'C008', 'C013', 'C018', 'C023', 'C024', 'C027', 'C037', 'C046', 'C047', 'C051', 'C053', 'C055', 'C056', 'C061', 'C062', 'C069', 'C073', 'C082', 'C083', 'C087', 'C090', 'C094', 'C097', 'C107'], 'snow_depth_mm': ['C053', 'C073'], 'avg_wind_dir_deg': ['C001', 'C004', 'C008', 'C034', 'C037', 'C043', 'C047', 'C049', 'C053', 'C055', 'C058', 'C062', 'C073', 'C077', 'C079', 'C087', 'C107', 'C108'], 'avg_wind_speed_kmh': ['C001', 'C004', 'C005', 'C007', 'C008', 'C009', 'C011', 'C014', 'C016', 'C022', 'C034', 'C035', 'C037', 'C042', 'C043', 'C046', 'C047', 'C048', 'C049', 'C053', 'C054', 'C055', 'C057', 'C058', 'C059', 'C062', 'C065', 'C068', 'C070', 'C071', 'C073', 'C077', 'C078', 'C079', 'C081', 'C082', 'C083', 'C085', 'C087', 'C089', 'C096', 'C099', 'C103', 'C105', 'C106', 'C107', 'C108', 'C109']}


In [11]:
for city in dataframe_dict:
    if city not in feature_df_dict['precipitation_mm']:
        dataframe_dict[city] = dataframe_dict[city].drop(columns=['precipitation_mm'])
    if city not in feature_df_dict['snow_depth_mm']:
        dataframe_dict[city] = dataframe_dict[city].drop(columns=['snow_depth_mm'])
    if city not in feature_df_dict['avg_wind_dir_deg']:
        dataframe_dict[city] = dataframe_dict[city].drop(columns=['avg_wind_dir_deg'])
    if city not in feature_df_dict['avg_wind_speed_kmh']:
        dataframe_dict[city] = dataframe_dict[city].drop(columns=['avg_wind_speed_kmh'])

In [12]:
null_per = {}
for city in dataframe_dict:
    null_per[city] = {}
    for col in dataframe_dict[city].columns:
        null_per[city][col] = dataframe_dict[city][col].isnull().sum()/dataframe_dict[city].shape[0]

In [13]:
for city in null_per:
    print(city,null_per[city])

C001 {'city_id': 0.0, 'date': 0.0, 'avg_temp_c': 0.0, 'min_temp_c': 0.001095290251916758, 'max_temp_c': 0.0016429353778751369, 'avg_wind_dir_deg': 0.011500547645125958, 'avg_wind_speed_kmh': 0.004381161007667032}
C002 {'city_id': 0.0, 'date': 0.0, 'avg_temp_c': 0.0, 'min_temp_c': 0.1064891846921797, 'max_temp_c': 0.5967831392124238}
C003 {'city_id': 0.0, 'date': 0.0, 'avg_temp_c': 0.0, 'min_temp_c': 0.06527701590784421, 'max_temp_c': 0.05924300603400987}
C004 {'city_id': 0.0, 'date': 0.0, 'avg_temp_c': 0.000547645125958379, 'min_temp_c': 0.000547645125958379, 'max_temp_c': 0.000547645125958379, 'precipitation_mm': 0.002738225629791895, 'avg_wind_dir_deg': 0.018619934282584884, 'avg_wind_speed_kmh': 0.00547645125958379}
C005 {'city_id': 0.0, 'date': 0.0, 'avg_temp_c': 0.0, 'min_temp_c': 0.006024096385542169, 'max_temp_c': 0.01533406352683461, 'avg_wind_speed_kmh': 0.02628696604600219}
C007 {'city_id': 0.0, 'date': 0.0, 'avg_temp_c': 0.0, 'min_temp_c': 0.001095890410958904, 'max_temp_c':

In [14]:
for city in dataframe_dict:
    dataframe_dict[city]['avg_temp_c'] = dataframe_dict[city]['avg_temp_c'].fillna((dataframe_dict[city]['max_temp_c'] + dataframe_dict[city]['min_temp_c']) / 2)
    dataframe_dict[city]['max_temp_c'] = dataframe_dict[city]['max_temp_c'].fillna(2 * dataframe_dict[city]['avg_temp_c'] - dataframe_dict[city]['min_temp_c'])
    dataframe_dict[city]['min_temp_c'] = dataframe_dict[city]['min_temp_c'].fillna(2 * dataframe_dict[city]['avg_temp_c'] - dataframe_dict[city]['max_temp_c'])

In [15]:
for city in dataframe_dict:
    dataframe_dict[city]['avg_temp_c'] = dataframe_dict[city]['avg_temp_c'].fillna(method='ffill')
    dataframe_dict[city]['max_temp_c'] = dataframe_dict[city]['max_temp_c'].fillna(dataframe_dict[city]['avg_temp_c'])
    dataframe_dict[city]['min_temp_c'] = dataframe_dict[city]['min_temp_c'].fillna(dataframe_dict[city]['avg_temp_c'])

/tmp/ipykernel_18/1745748021.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe_dict[city]['avg_temp_c'] = dataframe_dict[city]['avg_temp_c'].fillna(method='ffill')


In [16]:
for city in feature_df_dict['precipitation_mm']:
    dataframe_dict[city]['precipitation_mm'] = dataframe_dict[city]['precipitation_mm'].fillna(method='ffill')

for city in feature_df_dict['snow_depth_mm']:
    dataframe_dict[city]['snow_depth_mm'] = dataframe_dict[city]['snow_depth_mm'].fillna(method='ffill')

for city in feature_df_dict['avg_wind_dir_deg']:
    dataframe_dict[city]['avg_wind_dir_deg'] = dataframe_dict[city]['avg_wind_dir_deg'].fillna(method='ffill')

for city in feature_df_dict['avg_wind_speed_kmh']:
    dataframe_dict[city]['avg_wind_speed_kmh'] = dataframe_dict[city]['avg_wind_speed_kmh'].fillna(method='ffill')
    
for city in dataframe_dict:
    dataframe_dict[city] = dataframe_dict[city].dropna(axis=1, how='any')

/tmp/ipykernel_18/1176141871.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe_dict[city]['precipitation_mm'] = dataframe_dict[city]['precipitation_mm'].fillna(method='ffill')
/tmp/ipykernel_18/1176141871.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe_dict[city]['snow_depth_mm'] = dataframe_dict[city]['snow_depth_mm'].fillna(method='ffill')
/tmp/ipykernel_18/1176141871.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe_dict[city]['avg_wind_dir_deg'] = dataframe_dict[city]['avg_wind_dir_deg'].fillna(method='ffill')
/tmp/ipykernel_18/1176141871.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data

In [17]:
for city in dataframe_dict:
    dataframe_dict[city].index = pd.to_datetime(dataframe_dict[city]['date'])

In [18]:
city_model_features = {}
for city in dataframe_dict:
    city_model_features[city] = []
    for col in dataframe_dict[city].columns:
        if col not in ['city_id','date','min_temp_c','max_temp_c']:
            city_model_features[city].append(col)

In [19]:
final_df = {}
for city in dataframe_dict:
    final_df[city] = dataframe_dict[city][city_model_features[city]]


In [20]:
from sklearn.preprocessing import MinMaxScaler

def prepareDF(df_final):
    # Normalize the data
    labelData = df_final
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df_final)

    # Define sequence length and features
    sequence_length = 10  # Number of time steps in each sequence
    num_features = len(df_final.columns)

    # Create sequences and corresponding labels
    sequences = []
    labels = []
    for i in range(scaled_data.shape[0] - (sequence_length)):
        seq = scaled_data[i:i+sequence_length]
        label = labelData['avg_temp_c'][i+sequence_length]
        sequences.append(seq)
        labels.append(label)

    # Split into train and test sets
    train_size = int(0.8 * len(sequences))
    train_x, test_x = sequences[:train_size], sequences[train_size:]
    train_y, test_y = labels[:train_size], labels[train_size:]

    return train_x, train_y, test_x, test_y


In [21]:
train_x, train_y, test_x, test_y = prepareDF(final_df['C001'])


/tmp/ipykernel_18/4262586102.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = labelData['avg_temp_c'][i+sequence_length]


In [22]:
train_x

[array([[0.25867508, 0.46927374, 0.16666667],
        [0.34384858, 0.43296089, 0.33333333],
        [0.29022082, 0.43296089, 0.14912281],
        [0.29022082, 0.81284916, 0.39035088],
        [0.32176656, 0.81284916, 0.11403509],
        [0.33753943, 0.43575419, 0.14473684],
        [0.32807571, 0.44134078, 0.16666667],
        [0.32807571, 0.44134078, 0.16666667],
        [0.36277603, 0.44134078, 0.10526316],
        [0.39116719, 0.41620112, 0.14912281]]),
 array([[0.34384858, 0.43296089, 0.33333333],
        [0.29022082, 0.43296089, 0.14912281],
        [0.29022082, 0.81284916, 0.39035088],
        [0.32176656, 0.81284916, 0.11403509],
        [0.33753943, 0.43575419, 0.14473684],
        [0.32807571, 0.44134078, 0.16666667],
        [0.32807571, 0.44134078, 0.16666667],
        [0.36277603, 0.44134078, 0.10526316],
        [0.39116719, 0.41620112, 0.14912281],
        [0.37223975, 0.41620112, 0.14912281]]),
 array([[0.29022082, 0.43296089, 0.14912281],
        [0.29022082, 0.8128491

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def createModel(train_x):

    model = Sequential()

        # Add LSTM layers with dropout
    model.add(LSTM(units=128, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=32, return_sequences=False))
    model.add(Dropout(0.2))

        # Add a dense output layer
    model.add(Dense(units=1))

        # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


2024-06-22 06:03:19.457897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 06:03:19.458047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 06:03:19.615950: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [24]:
def train_model(train_x,train_y,model):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    # Train the models
    
    history = model.fit(
        train_x, train_y,
        epochs=200,
        batch_size=64,
        validation_split=0.2,  # Use part of the training data as validation
        callbacks=[early_stopping]
    )
    return model

In [25]:
model_C001 = train_model(train_x,train_y,createModel(train_x))

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Assuming you have trained the model and have the 'best_model' object
# Also, 'test_x' and 'test_y' should be available

# Predict temperatures using the trained model
predictions = C112_model.predict(test_x)

# Calculate evaluation metrics
mae = mean_absolute_error(test_y, predictions)
mse = mean_squared_error(test_y, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
city_models={}
for city in final_df:
    train_x, train_y = prepareDF(final_df[city])
    model = createModel(train_x)
    train_model(train_x,train_y,model)
    city_models[city] = model

In [ ]:
def get_final_X(df_final):
    # Normalize the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df_final.values.reshape(-1, 1))

    # Define sequence length
    sequence_length = 10  # Number of time steps in each sequence

    # Get the final sequence of 10 values
    final_sequence = scaled_data[-sequence_length:]

    return final_sequence, scaler

def prediction(model, df):
    final_seq, scaler = get_final_X(df)
    final_seq = final_seq.reshape((1, 10, 1)).astype(np.float32)  # Reshape and ensure dtype is float32

    predictions = []
    for _ in range(7):
        # Predict the next value
        new_value = model.predict(final_seq)
        new_value = new_value[0, 0]  # Extract the predicted value

        # Append the prediction
        predictions.append(new_value)

        # Normalize the new predicted value
        new_value_scaled = scaler.transform([[new_value]])[0, 0]

        # Update the sequence
        final_seq = np.append(final_seq[0, 1:], [[new_value_scaled]], axis=0).reshape((1, 10, 1)).astype(np.float32)  # Update and reshape

    # Inverse transform predictions to original scale
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    return predictions.flatten()

In [ ]:
pred = {}
for city in final_df:
    pred[city] = prediction(city_models[city],final_df[city])


In [ ]:
pred

In [ ]:
values = []
for city in pred:
    for val in pred[city]:
        values.append(val[0][0])
series = pd.Series(values)
    

In [ ]:
print(series)

In [ ]:
series.to_csv('mycsv.csv', index=False)